In [1]:
from langchain.llms import CTransformers
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.251
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\devenderjhinkwan\ml_path\effective_price_amazon_flipkart\myenv\lib\site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [29]:
#pip install langchain==0.0.251

In [11]:
#pip install ctransformers==0.2.5

In [13]:
#pip install sentence-transformers==2.2.2

In [2]:
# Local CTransformers wrapper for Llama-2-7B-Chat
llm = CTransformers(model='models/llama-2-7b-chat.ggmlv3.q8_0.bin', # Location of downloaded GGML model
                    model_type='llama', # Model type Llama
                    config={'max_new_tokens': 256,
                            'temperature': 0.01})

In [3]:
llm

CTransformers(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<ctransformers.llm.LLM object at 0x00000184010FFB50>, model='models/llama-2-7b-chat.ggmlv3.q8_0.bin', model_type='llama', model_file=None, config={'max_new_tokens': 256, 'temperature': 0.01}, lib=None)

#### Zero Shot Inference with Instruction prompting

In [5]:
from langchain import PromptTemplate, LLMChain

template = """Calculate the maximum discount out of all the offers: {scenario}

Maximum Discount:"""

prompt = PromptTemplate(template=template, input_variables=["scenario"])


In [13]:
prompt_zero_shot = f"""

The Price of SKU is 15899

Offers:
1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.
2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.
3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999
4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999

what is the maximum discount? 
"""

In [14]:
prompt_zero_shot

'\n\nThe Price of SKU is 15899\n\nOffers:\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999\n\nwhat is the maximum discount? \n'

In [15]:

llm_chain = LLMChain(prompt=prompt, llm=llm)

response = llm_chain.run(prompt_zero_shot)

In [16]:
response

' ₹750\n\nExplanation:\nThe maximum discount that can be availed is ₹750, which is offered by Offer 4. This offer provides a flat ₹750 instant discount on OneCard Credit Card non-EMI transactions with a minimum purchase value of ₹15999. As the price of SKU is ₹15899, which is less than the minimum purchase value required for this offer, the maximum discount that can be availed is ₹750.'

#### One shot Inference

In [6]:
template_oneshot = """You are an AI assistant which helps in evaluating the offers for maximum discount. Considering the SKU price & offers listed calculate the maximum discount applicable \n\n: {scenario}

Maximum Discount: {solution}

"""

prompt_oneshot = PromptTemplate(template=template, input_variables=["scenario"])


In [7]:
example = {"scenario":"The Price of SKU is 17999. The below offers are applicable:\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase 4. value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999.\n What is the maximum discount?",
          "solution":"Assuming prime member from the first offer the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of price of sku, which is 17999. This equals 900. From the second offer the maximum discount is 100 cashback + 500 welcome rewards= 600. so the discount from this offer is 600. From the third offer since the purchase value 17999 is greater than Minimum purchase value of ₹15999, hence the discount amount is applicable & it is equal to ₹500. From the fourth offer the discount = ₹750 since the Minimum pucrhase value of ₹15999 is less than than ₹17999. Comparing the discounts from the 4 offers the maximum discount is ₹900. So the answer is ₹900"}

In [8]:
# Make the prompt

def make_prompt(example, example_to_solve):
    prompt = ''
    scenario = example['scenario']
    solution = example['solution']
    
    prompt += f"""Calculate the maximum discount out of all the offers: \n\n{scenario}
    
    Maximum Discount: {solution}
    
    """

    #print(prompt)

    scenario = example_to_solve['scenario']
    
    prompt += f"""Calculate the maximum discount out of all the offers: \n\n{scenario}
    
    Maximum Discount: """
    
    return prompt

In [9]:
example_to_solve = {"scenario": "The Price of SKU is 15899. The below offers are applicable:\n1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.\n2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.\n3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase value INR 15999\n4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999.\What is the maximum discount?"}

one_shot_prompt = make_prompt(example, example_to_solve)

print(one_shot_prompt)

Calculate the maximum discount out of all the offers: 

The Price of SKU is 17999. The below offers are applicable:
1. Get 5% back with Amazon Pay ICICI Bank credit card for Prime members. 3% back for others. Not applicable on Amazon business transactions.
2. ₹100 cashback & ₹500 welcome rewards on Amazon Pay Later. Activate now. Pay next month at Zero interest or in EMIs! T&C apply.
3. Flat INR 500 Instant Discount on ICICI Bank Credit Cards (excluding Amazon Pay ICICI Credit Card) Credit Card Txn. Minimum purchase 4. value INR 15999
4. Flat INR 750 Instant Discount on OneCard Credit Card Non EMI Txn. Minimum purchase value INR 15999.
 What is the maximum discount?
    
    Maximum Discount: Assuming prime member from the first offer the maximum discount applicable is 5%, since 5% is greater than 3%. So the discount amount is 5% of price of sku, which is 17999. This equals 900. From the second offer the maximum discount is 100 cashback + 500 welcome rewards= 600. so the discount from 

In [10]:
llm_chain = LLMChain(prompt=prompt_oneshot, llm=llm)

response = llm_chain.run(one_shot_prompt)
print(response)

 Assuming prime member from First Offer: Assuming Prime Member: Assuming prime member from first offer  Calculation: Assuming prime member from the customer can be applicable for this offer ₹5% = 5%: Assuming prime member from the maximum discount: Assuming prime member from the customer can be applicable is 
     Discount: Assuming prime member from First Offer: Assuming Prime Member: Assuming prime member from first offer  Calculation: Assuming prime member from the customer can be applicable for this offer ₹90: Assuming prime member from the customer can be applicable is 5% = 5%: Assuming prime member from the maximum discount: Assuming prime member from First Offer: Assuming Prime Member Discount: Assuming prime member from first offer 
     | Offer: Assuming prime member from the customer can be applicable for this offer  Calculation: Assuming prime member from the customer can be applicable is 5% = ₹90: Assuming prime member from the maximum discount: Assuming prime member from F